## **Исходные данные:**

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).<br>

Данные представлены в формате CSV (Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа)).  Каждая строка представляет молекулу.<br>

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1];
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.<br>


Предварительная обработка не требуется, данные уже закодированы и нормализованы.<br>

В качестве метрики будем использовать **F1-score**.<br>

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

## **Решение:**

In [2]:
# Подключаем библиотеки
from google.colab import drive
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных
import matplotlib.pyplot as plt # для визуализации
import seaborn as sns # для визуализации

from sklearn import linear_model # линейные моделиё
from sklearn import tree # деревья решений
from sklearn import ensemble # ансамбли
from sklearn import metrics # метрики
from sklearn import preprocessing # предобработка
from sklearn.model_selection import train_test_split # сплитование выборки

# Импорт библиотек для оптимизации гиперпараметров
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
!pip install optuna
import optuna





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00


#### **1. Загрузка и обработка данных**

In [20]:
# Монтируем Google Диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Загружаем данные
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Andrew_learning/_train_sem09 (1).csv')
df.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Размерность датафрейма
df.shape

(3751, 1777)

**Проверим датафрейм на наличие пропусков и дубликатов, а также на корректность типов данных столбцов**

In [23]:
# Выводим основную информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3751 entries, 0 to 3750
Columns: 1777 entries, Activity to D1776
dtypes: float64(942), int64(835)
memory usage: 50.9 MB


In [24]:
# Выполним проверку на наличие пропущенных значений NaN в датафрейме
df.isna().sum()

,0
Activity,0
D1,0
D2,0
D3,0
D4,0
...,...
D1772,0
D1773,0
D1774,0
D1775,0


Для обработки пропусков воспользуемся **методом fillna**

In [25]:
# Проверка на наличие дубликатов
dupl_columns = list(df.columns) # создаем список столбцов dupl_columns, по которым будем искать совпадения
# Создаем маску дубликатов с помощью метода duplicated() и произведём фильтрацию
mask = df.duplicated(subset=dupl_columns)
df_duplicates = df[mask]
print(f'Количество найденных дубликатов: {df_duplicates.shape[0]}')

Количество найденных дубликатов: 0


In [26]:
# Выведим строки, которые являются полными дубликатами, т.е совпадают по всем столбцам
duplicates = df[df.duplicated()]
print("Строки, которые являются полными дубликатами:")
print(duplicates)

Строки, которые являются полными дубликатами:
Empty DataFrame
Columns: [Activity, D1, D2, D3, D4, D5, D6, D7, D8, D9, D10, D11, D12, D13, D14, D15, D16, D17, D18, D19, D20, D21, D22, D23, D24, D25, D26, D27, D28, D29, D30, D31, D32, D33, D34, D35, D36, D37, D38, D39, D40, D41, D42, D43, D44, D45, D46, D47, D48, D49, D50, D51, D52, D53, D54, D55, D56, D57, D58, D59, D60, D61, D62, D63, D64, D65, D66, D67, D68, D69, D70, D71, D72, D73, D74, D75, D76, D77, D78, D79, D80, D81, D82, D83, D84, D85, D86, D87, D88, D89, D90, D91, D92, D93, D94, D95, D96, D97, D98, D99, ...]
Index: []

[0 rows x 1777 columns]


**Выводы:** Как видно из проведенного анализа в датафрейме отсутствуют пропущенные значения и дубликаты, ввиду того, что согласно исходным данным предварительная обработка датафрейма не требуется, так как данные уже закодированы и нормализованы.

#### **2. Разделение данных на тренировочные и тестовые**

In [27]:
# Разделение данных на признаки (X) и целевую переменную (y)
X = df.drop(columns='Activity')
y = df['Activity']

# Разделение на тренировочную и тестовую выборки (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Размеры полученных выборок
split_info = {
    "X_train_shape": X_train.shape,
    "X_test_shape": X_test.shape,
    "y_train_distribution": y_train.value_counts(),
    "y_test_distribution": y_test.value_counts(),
}
split_info

{'X_train_shape': (3000, 1776),
 'X_test_shape': (751, 1776),
 'y_train_distribution': Activity
 1    1627
 0    1373
 Name: count, dtype: int64,
 'y_test_distribution': Activity
 1    407
 0    344
 Name: count, dtype: int64}

Тренировочная выборка: 3000 наблюдений (1627 - класса 1, 1373 - класса 0).<br>
Тестовая выборка: 751 наблюдений (407 - класса 1, 344 - класса 0).

In [28]:
# Проверим, насколько равномерно разделены целевые данные
y.value_counts(normalize=True)

,proportion
Activity,
1,0.542255
0,0.457745


Данные разделены примерно одинаково, поэтому делать стратификацию не обязательно

#### **3. Обучение моделей (логистическая регрессия и случайный лес)**

Рассчитаем F1-score, построив базовую модель машинного обучения с использованием логистической регрессии

In [29]:
logReg_base = linear_model.LogisticRegression(random_state=42, max_iter= 50)

logReg_base.fit(X_train, y_train)

y_pred_logReg_base = logReg_base.predict(X_test)

f1_logReg_base = metrics.f1_score(y_test, y_pred_logReg_base)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
print('F1-score для базовой модели с использованием логистической регресии равен {:.2f}'.format(f1_logReg_base))

F1-score для базовой модели с использованием логистической регресии равен 0.78


Рассчитаем F1-score, построив базовую модель машинного обучения с использованием алгоритма случайного леса (Random Forest)

In [31]:
rf_base = ensemble.RandomForestClassifier(random_state= 42)

rf_base.fit(X_train, y_train)

y_pred_rf_base = rf_base.predict(X_test)

f1_rf_base = metrics.f1_score(y_test, y_pred_rf_base)

In [32]:
print('F1-score для базовой модели с использованием алгоритма случайного леса равен {:.2f}'.format(f1_rf_base))

F1-score для базовой модели с использованием алгоритма случайного леса равен 0.81


**Вывод:**<br>
В результате выполненных расчетов:<br>
F1-метрика (F1-score) c использованием логистической регрессия равна: 0.78.<br>
F1-метрика (F1-score) c использованием алгоритма случайнго леса равна: 0.81.<br>
Как видно, случайный лес показал немного лучшие результаты.<br>
Далее выполним подбор гиперпараметров для обеих моделей.

#### **4. Подбор гиперпараметров c использованием методов: GridSearchCV, RandomizedSearchCV, Hyperopt, Optuna**


##### **Построение модели с помощью GridSearchCV**


*на основе логистической регресии:*

In [ ]:
param_grid = [
    {'penalty' : ['l2', 'none'], # тип регуляризации
    'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
    'C' : [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации

    # так как разные алгоритмы поддерживают разные типы регуляризации, то создадим еще 1 набор параметров
    {'penalty': ['l1', 'l2'] ,
    'solver': ['liblinear', 'saga'],
    'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

gs_logReg = GridSearchCV(
    estimator = linear_model.LogisticRegression(random_state= 42, max_iter = 50),
    param_grid = param_grid,
    cv = 5,
    n_jobs = -1,
    scoring = 'f1'
)

%time gs_logReg.fit(X_train, y_train)

y_pred_gs_logReg = gs_logReg.predict(X_test)

f1_gs_logReg = metrics.f1_score(y_test, y_pred_gs_logReg)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
105 fits failed out of a total of 350.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
105 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

CPU times: user 11.8 s, sys: 907 ms, total: 12.7 s
Wall time: 8min 35s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
print("Наилучшие значения гиперпараметров: {}".format(gs_logReg.best_params_))

Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


In [ ]:
print('F1-score на GridSearchCV для логистической регресии равен {:.2f}'.format(f1_gs_logReg))

F1-score на GridSearchCV для логистической регресии равен 0.79


*на основе случайного леса:*

In [ ]:
param_grid = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': list(np.linspace(5, 25, 10, dtype=int)),
              'max_depth': list(np.linspace(1, 30, 6, dtype=int))
              }

gs_rf = GridSearchCV(
    estimator = ensemble.RandomForestClassifier(random_state= 42),
    param_grid = param_grid,
    cv = 5,
    n_jobs = -1,
    scoring = 'f1'
)

%time gs_rf.fit(X_train, y_train)

f1_gs_rf = gs_rf.score(X_test, y_test)

CPU times: user 15.3 s, sys: 2.09 s, total: 17.4 s
Wall time: 20min 19s


In [ ]:
print("Наилучшие значения гиперпараметров: {}".format(gs_rf.best_params_))

Наилучшие значения гиперпараметров: {'max_depth': 18, 'min_samples_leaf': 5, 'n_estimators': 170}


In [ ]:
print('F1-score на GridResearchCV для случайного леса равен {:.2f}'.format(f1_gs_rf))

F1-score на GridResearchCV для случайного леса равен 0.80


##### **Построение модели с помощью RandomizedSearchCV**


*на основе логистической регресии:*

In [ ]:
param_random = [
    {'penalty' : ['l2', 'none'], # тип регуляризации
    'solver' : ['newton-cg', 'lbfgs', 'sag'], # алгоритм оптимизации
    'C': list(np.linspace(0.01, 1, 10, dtype=float))}, # уровень силы регурялизации

    # так как разные алгоритмы поддерживают разные типы регуляризации, то создадим еще 1 набор параметров
    {'penalty': ['l1', 'l2'] ,
    'solver': ['liblinear', 'saga'],
    'C': list(np.linspace(0.01, 1, 10, dtype=float))}
]

rs_logReg = RandomizedSearchCV(
    estimator= linear_model.LogisticRegression(random_state=42, max_iter=50),
    param_random=param_random,
    cv = 5,
    n_iter = 50,
    n_jobs = -1,
    scoring = 'f1'
)

%time rs_logReg.fit(X_train, y_train)

f1_rs_logReg = rs_logReg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
60 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

CPU times: user 4.12 s, sys: 696 ms, total: 4.82 s
Wall time: 7min 31s


In [ ]:
print("Наилучшие значения гиперпараметров: {}".format(rs_logReg.best_params_))

Наилучшие значения гиперпараметров: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.45}


In [ ]:
print('F1-score на RandomizedSrearchCV для логистической регресии равен {:.2f}'.format(f1_rs_logReg))

F1-score на RandomizedSrearchCV для логистической регресии равен 0.78


*на основе случайного леса:*

In [ ]:
param_random = {
    'min_samples_leaf' : list(np.linspace(5, 25, 10, dtype=int)),
    'max_depth' : list(np.linspace(1, 30, 6, dtype = int)),
    'n_estimators' : list(range(80, 200, 30))
}

rs_rf = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_random=param_random,
    cv = 5,
    n_iter = 50,
    n_jobs = -1,
    scoring = 'f1'
)

%time rs_rf.fit(X_train, y_train)

f1_rs_rf = rs_rf.score(X_test, y_test)

CPU times: user 4.76 s, sys: 486 ms, total: 5.24 s
Wall time: 4min 9s


In [ ]:
print("Наилучшие значения гиперпараметров: {}".format(rs_rf.best_params_))

Наилучшие значения гиперпараметров: {'n_estimators': 140, 'min_samples_leaf': 5, 'max_depth': 18}


In [ ]:
print('F1-score на RandomizedSrearchCV для случайного леса равен {:.2f}'.format(f1_rs_rf))

F1-score на RandomizedSrearchCV для случайного леса равен 0.80


##### **Построение модели с помощью HyperOpt**


*на основе логистической регресии:*

In [199]:
# Создаем пространство гиперпараметров
space = {
    'penalty': hp.choice(label='penalty', options=['l1', 'l2']),  # тип регуляризации
    'solver': hp.choice(label='solver', options=['liblinear', 'saga']),  # алгоритм оптимизации
    'C': hp.loguniform(label='C', low=-2 * np.log(10), high=2 * np.log(10))  # уровень силы регуляризации
}

random_state = 42 # зафиксируем random_state

def hyperopt_lr(params, cv=5, X=None, y=None, random_state=random_state):

    # Инициализация модели с переданными параметрами
    model = linear_model.LogisticRegression(
        penalty=params['penalty'],
        solver=params['solver'],
        C=params['C'],
        random_state=random_state,
        max_iter=50
    )

    # Применим кросс-валидацию
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # Для минимизации F1-метрики ставим знак минус
    return -score

# Логирование результатов
trials = Trials()

# Запуск оптимизации с Hyperopt
best = fmin(fn=lambda params: hyperopt_lr(params, X=X_train, y=y_train),
            space=space,  # пространство гиперпараметров
            algo=tpe.suggest,  # алгоритм оптимизации
            max_evals=50,  # количество итераций
            trials=trials,  # логирование результатов
            rstate=np.random.default_rng(random_state))  # генератор случайных чисел

print(f'Лучшие гиперпараметры: {best}')

100%|██████████| 50/50 [15:29<00:00, 18.59s/trial, best loss: -0.7928648789728487]
Лучшие гиперпараметры: {'C': 0.020162206477661048, 'penalty': 1, 'solver': 1}


In [207]:
# Находим наилучшую best_loss (отрицательный F1-score для лучших гиперпараметров) и преобразуем обратно в F1-score
best_loss = min(trials.losses())
best_f1_score = -best_loss  # так как мы минимизируем отрицательный F1

print("Наилучшие значения гиперпараметров: {}".format(best))
print(f"F1-score на HyperOpt для логистической регресии равен : {best_f1_score:.2f}")

Наилучшие значения гиперпараметров: {'C': 0.020162206477661048, 'penalty': 1, 'solver': 1}
F1-score на HyperOpt для логистической регресии равен : 0.79


*на основе случайного леса:*

In [208]:
# Создаем пространство гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 80, 200, 1),
       'max_depth' : hp.quniform('max_depth', 1, 30, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 5, 25, 1)
      }

In [209]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):

    params = {'n_estimators': int(params['n_estimators']),
              'max_depth': int(params['max_depth']),
             'min_samples_leaf': int(params['min_samples_leaf'])
    }
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # Для минимизации F1-метрики ставим знак минус
    return -score

In [212]:
%time

trials = Trials() # используется для логирования результатов

best = fmin(hyperopt_rf, # наша функция
          space = space, # пространство гиперпараметров
          algo = tpe.suggest, # алгоритм оптимизации
          max_evals = 50, # максимальное количество итераций
          trials = trials, # логирование результатов
          rstate=np.random.default_rng(random_state)  # фиксируем для повторяемости результата
)
print("Наилучшие значения гиперпараметров {}".format(best))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs
100%|██████████| 50/50 [08:07<00:00,  9.74s/trial, best loss: -0.8126347227817426]
Наилучшие значения гиперпараметров {'max_depth': 27.0, 'min_samples_leaf': 6.0, 'n_estimators': 198.0}


In [213]:
# Находим наилучшую best_loss (отрицательный F1-score для лучших гиперпараметров) и преобразуем обратно в F1-score
best_loss = min(trials.losses())
best_f1_score = -best_loss  # так как мы минимизируем отрицательный F1

print("Наилучшие значения гиперпараметров: {}".format(best))
print(f"F1-score на HyperOpt для случайного леса равен : {best_f1_score:.2f}")

Наилучшие значения гиперпараметров: {'max_depth': 27.0, 'min_samples_leaf': 6.0, 'n_estimators': 198.0}
F1-score на HyperOpt для случайного леса равен : 0.81


##### **Построение модели с помощью Optuna**


*на основе логистической регресии:*

In [39]:
def optuna_lr(trial):
    penalty = trial.suggest_categorical(name='penalty', choices= ['l1', 'l2']) # тип регуляризации
    solver = trial.suggest_categorical(name = 'solver', choices= ['liblinear', 'saga']) # алгоритм оптимизации
    C = trial.suggest_float(name='C', low=0.01, high=1, step = 0.1) # уровень силы регурялизации

    random_state = 42 # зафиксируем random_state

    model = linear_model.LogisticRegression(
        penalty = penalty,
        solver = solver,
        C = C,
        random_state=random_state
    )

    model.fit(X_train, y_train)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()

    return score

In [40]:
%%time

study = optuna.create_study(study_name='LogisticRegression', direction='maximize')

study.optimize(optuna_lr, n_trials= 50)

[I 2025-01-10 12:09:34,371] A new study created in memory with name: LogisticRegression
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:693: UserWarning: The distribution is specified by [0.01, 1] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.01, 0.91].
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-01-10 12:11:13,527] Trial 0 finished with value: 0.7850653891519912 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.6100000000000001}. Best is trial 0 with value: 0.7850653891519912.
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:693: UserWarning: The distribution is specified by [0.01, 1] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.01, 0.91].
  warnings.warn(
[I 2025-01-10 12:11:15,689] Trial 1 finished with value: 0.780997

CPU times: user 2min 3s, sys: 2.96 s, total: 2min 6s
Wall time: 11min 24s


In [42]:
# Выведем наилучшие значения гиперпараметров и F1-score
best_params_opt_lr = study.best_params
best_f1_score_opt_lr = study.best_value

print("Наилучшие значения гиперпараметров: {}".format(best_params_opt_lr))
print(f"F1-score на Optuna для логистической регрессии равен: {best_f1_score_opt_lr:.2f}")

Наилучшие значения гиперпараметров: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.21000000000000002}
F1-score на Optuna для логистической регрессии равен: 0.79


*на основе случайного леса:*

In [43]:
def optuna_rf(trial):
    n_estimators =  trial.suggest_int('n_estimators', 80, 200, 1)
    max_depth = trial.suggest_int('max_depth', 1, 30, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 25, 1)

    random_state = 42 # зафиксируем random_state

    model = ensemble.RandomForestClassifier(
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_samples_leaf = min_samples_leaf,
        random_state = random_state
    )

    model.fit(X_train, y_train)
    score = cross_val_score(model, X_train, y_train, cv = 5, scoring = 'f1', n_jobs= -1).mean()

    return score

In [44]:
%%time

study = optuna.create_study(study_name='RandomForestClassification', direction='maximize')

study.optimize(optuna_rf, n_trials= 50)

[I 2025-01-10 12:24:18,088] A new study created in memory with name: RandomForestClassification
<ipython-input-43-3c134b65791b>:2: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators =  trial.suggest_int('n_estimators', 80, 200, 1)
<ipython-input-43-3c134b65791b>:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  max_depth = trial.suggest_int('max_depth', 1, 30, 1)
<ipython-input-43-3c134b65791b>:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 25, 1)
[I 2025-01-10 12:24:25,412] Trial 0 finished with value: 0.782274892836979 and parameters: {'n_estimators': 144, 'max_depth': 9, 'min_samples_leaf': 24}. Best is trial 0 with value: 0.782274892836979.
<ipython-input-43-3c134b65791b>:2: Future

CPU times: user 1min 49s, sys: 2.12 s, total: 1min 52s
Wall time: 8min 7s


In [46]:
# Выведем наилучшие значения гиперпараметров и F1-score

best_params_opt_rf = study.best_params
best_f1_score_opt_rf = study.best_value

print("Наилучшие значения гиперпараметров: {}".format(best_params_opt_rf))
print(f"F1-score на Optuna для логистической регрессии равен: {best_f1_score_opt_rf:.2f}")


Наилучшие значения гиперпараметров: {'n_estimators': 101, 'max_depth': 14, 'min_samples_leaf': 6}
F1-score на Optuna для логистической регрессии равен: 0.81


In [47]:
optuna.visualization.plot_optimization_history(study, target_name="F1-score")

#### **5. Оценка метрики F1-score и сравнение результатов**

В результате выполненных расчетов получили следующие данные:<br>
1. Для базовой модели: <br>
F1-метрика (F1-score) c использованием логистической регрессия равна: 0.78.<br>
F1-метрика (F1-score) c использованием алгоритма случайнго леса равна: 0.81.<br>
2. Построение модели с помощью GridSearchCV:<br>
F1-метрика (F1-score) c использованием логистической регрессия равна: 0.79.<br>
F1-метрика (F1-score) c использованием алгоритма случайнго леса равна: 0.80.<br>
3. Построение модели с помощью RandomizedSearchCV:<br>
F1-метрика (F1-score) c использованием логистической регрессия равна: 0.78.<br>
F1-метрика (F1-score) c использованием алгоритма случайнго леса равна: 0.80.<br>
4. Построение модели с помощью HyperOpt:<br>
F1-метрика (F1-score) c использованием логистической регрессия равна: 0.79.<br>
F1-метрика (F1-score) c использованием алгоритма случайнго леса равна: 0.81.<br>
5. Построение модели с помощью Optuna:<br>
F1-метрика (F1-score) c использованием логистической регрессия равна: 0.79.<br>
F1-метрика (F1-score) c использованием алгоритма случайнго леса равна: 0.81.<br>